# 行转列(主要是unstack/pivot的应用)

In [2]:
import pandas as pd
import numpy as np
from functools import reduce

In [66]:
# 行转列
# 行转列则是把一张高表变成一张宽表
data = [['古明地觉', '语文', 90],
['古明地觉', '数学', 95],
['古明地觉', '英语', 96],
['芙兰朵露', '语文', 87],
['芙兰朵露', '数学', 92],
['芙兰朵露', '英语', 98],
['琪露诺',  '语文', 100],
['琪露诺',  '数学', 9],
['琪露诺',  '英语', 91]]
columns_list = ['name', 'subject', 'score']

row_col_df = pd.DataFrame(data, columns=columns_list)
print(row_col_df)

   name subject  score
0  古明地觉      语文     90
1  古明地觉      数学     95
2  古明地觉      英语     96
3  芙兰朵露      语文     87
4  芙兰朵露      数学     92
5  芙兰朵露      英语     98
6   琪露诺      语文    100
7   琪露诺      数学      9
8   琪露诺      英语     91


In [4]:
# # 将name和subject置为索引, 然后取出score这一列, 得到的对应的具有二级索引的Series对象
two_level_index_series = row_col_df.set_index(['name', 'subject'])['score']
print(two_level_index_series)

name  subject
古明地觉  语文          90
      数学          95
      英语          96
芙兰朵露  语文          87
      数学          92
      英语          98
琪露诺   语文         100
      数学           9
      英语          91
Name: score, dtype: int64


In [7]:
# unstack 将行转列
new_df = two_level_index_series.unstack()
print(new_df)

subject  数学  英语   语文
name                
古明地觉     95  96   90
琪露诺       9  91  100
芙兰朵露     92  98   87


In [12]:
# 将行索引subject去掉
new_df.rename_axis(mapper=None, axis=1, inplace=True)
print(new_df)
new_df = new_df.reset_index()

      数学  英语   语文
name             
古明地觉  95  96   90
琪露诺    9  91  100
芙兰朵露  92  98   87


In [11]:
new_df.reset_index()

,name,数学,英语,语文
0,古明地觉,95,96,90
1,琪露诺,9,91,100
2,芙兰朵露,92,98,87


In [13]:
# 调用unstack默认是将一级索引变成DataFrame的索引，二级索引变成DataFrame的列。
# 更准确的说，unstack是将最后一级的索引变成DataFrame的列，前面的索引变成DataFrame的索引。
# 比如有一个具有八级索引的Series，它在调用unstack的时候，默认是将最后一级索引变成DataFrame的列，前面七个索引整体作为DataFrame的索引。

# 只不过索引一般很少有超过二级的，所以这里就用二级举例了。因此问题来了，那么可不可以将一级索引(这里的"姓名")变成DataFrame的列，
# 二级索引(这里的"科目")变成DataFrame的行呢？答案是可以的，在unstack中指定一个参数即可

# 这里的level默认是-1, 表示将最后一级的索引变成列
# 这里我们指定为0(注意: 索引从0开始), 告诉pandas, 把第一级索引变成列
two_level_index_series.unstack(level=0).rename_axis(mapper=None, axis=1).reset_index()

,subject,古明地觉,琪露诺,芙兰朵露
0,数学,95,9,92
1,英语,96,91,98
2,语文,90,100,87


In [14]:
# pivot 实现行转列
row_col_df.pivot(index='name', columns='subject', values='score')

subject,数学,英语,语文
name,,,
古明地觉,95,96,90
琪露诺,9,91,100
芙兰朵露,92,98,87


In [17]:
row_col_df.pivot(index='name', columns='subject', values='score').rename_axis(mapper=None, axis=1).reset_index()

,name,数学,英语,语文
0,古明地觉,95,96,90
1,琪露诺,9,91,100
2,芙兰朵露,92,98,87


In [16]:
row_col_df

,name,subject,score
0,古明地觉,语文,90
1,古明地觉,数学,95
2,古明地觉,英语,96
3,芙兰朵露,语文,87
4,芙兰朵露,数学,92
5,芙兰朵露,英语,98
6,琪露诺,语文,100
7,琪露诺,数学,9
8,琪露诺,英语,91


# 一行转为多行(主要是unstack|explode的应用)

In [20]:
data_list = [['琪亚娜·卡斯兰娜', '12月7日','陶典,钉宫理惠'],
['布洛妮娅·扎伊切克', '8月18日','TetraCalyx,Hanser,阿澄佳奈,花泽香菜'],
['德丽莎·阿波卡利斯', '3月28日','花玲,田村由香里'],]

df = pd.DataFrame(data=data_list, columns=['name', 'birthday', 'voice'])

In [21]:
df

,name,birthday,voice
0,琪亚娜·卡斯兰娜,12月7日,"陶典,钉宫理惠"
1,布洛妮娅·扎伊切克,8月18日,"TetraCalyx,Hanser,阿澄佳奈,花泽香菜"
2,德丽莎·阿波卡利斯,3月28日,"花玲,田村由香里"


In [49]:
# 筛选出"voice"这个字段, 此时得到的是一个具有二级索引的Series
df.set_index(['name', 'birthday'])['voice'].str.split(",")

name       birthday
琪亚娜·卡斯兰娜   12月7日                             [陶典, 钉宫理惠]
布洛妮娅·扎伊切克  8月18日       [TetraCalyx, Hanser, 阿澄佳奈, 花泽香菜]
德丽莎·阿波卡利斯  3月28日                            [花玲, 田村由香里]
Name: voice, dtype: object

In [27]:
df.set_index(['name', 'birthday'])['voice'].str.split(",", expand=True)

,,0,1,2,3
name,birthday,,,,
琪亚娜·卡斯兰娜,12月7日,陶典,钉宫理惠,None,None
布洛妮娅·扎伊切克,8月18日,TetraCalyx,Hanser,阿澄佳奈,花泽香菜
德丽莎·阿波卡利斯,3月28日,花玲,田村由香里,None,None


In [48]:
df.set_index(['name', 'birthday'])['voice'].str.split(",", expand=True)\
        .stack()

name       birthday   
琪亚娜·卡斯兰娜   12月7日     0            陶典
                     1          钉宫理惠
布洛妮娅·扎伊切克  8月18日     0    TetraCalyx
                     1        Hanser
                     2          阿澄佳奈
                     3          花泽香菜
德丽莎·阿波卡利斯  3月28日     0            花玲
                     1         田村由香里
dtype: object

In [29]:
df.set_index(['name', 'birthday'])['voice'].str.split(",", expand=True)\
        .stack().reset_index()

,name,birthday,level_2,0
0,琪亚娜·卡斯兰娜,12月7日,0,陶典
1,琪亚娜·卡斯兰娜,12月7日,1,钉宫理惠
2,布洛妮娅·扎伊切克,8月18日,0,TetraCalyx
3,布洛妮娅·扎伊切克,8月18日,1,Hanser
4,布洛妮娅·扎伊切克,8月18日,2,阿澄佳奈
5,布洛妮娅·扎伊切克,8月18日,3,花泽香菜
6,德丽莎·阿波卡利斯,3月28日,0,花玲
7,德丽莎·阿波卡利斯,3月28日,1,田村由香里


In [47]:
# reset_index(drop=True, level=[-1]) 表示删除索引项的最后一个索引
df.set_index(['name', 'birthday'])['voice'].str.split(",", expand=True)\
        .stack().reset_index(drop=True, level=[-1]).reset_index().rename(columns={0: 'voice'})

,name,birthday,voice
0,琪亚娜·卡斯兰娜,12月7日,陶典
1,琪亚娜·卡斯兰娜,12月7日,钉宫理惠
2,布洛妮娅·扎伊切克,8月18日,TetraCalyx
3,布洛妮娅·扎伊切克,8月18日,Hanser
4,布洛妮娅·扎伊切克,8月18日,阿澄佳奈
5,布洛妮娅·扎伊切克,8月18日,花泽香菜
6,德丽莎·阿波卡利斯,3月28日,花玲
7,德丽莎·阿波卡利斯,3月28日,田村由香里


In [50]:
# explode应用
df

,name,birthday,voice
0,琪亚娜·卡斯兰娜,12月7日,"陶典,钉宫理惠"
1,布洛妮娅·扎伊切克,8月18日,"TetraCalyx,Hanser,阿澄佳奈,花泽香菜"
2,德丽莎·阿波卡利斯,3月28日,"花玲,田村由香里"


In [51]:
df['voice'] = df['voice'].str.split(",")

In [52]:
df

,name,birthday,voice
0,琪亚娜·卡斯兰娜,12月7日,"[陶典, 钉宫理惠]"
1,布洛妮娅·扎伊切克,8月18日,"[TetraCalyx, Hanser, 阿澄佳奈, 花泽香菜]"
2,德丽莎·阿波卡利斯,3月28日,"[花玲, 田村由香里]"


In [53]:
df.explode('voice')

,name,birthday,voice
0,琪亚娜·卡斯兰娜,12月7日,陶典
0,琪亚娜·卡斯兰娜,12月7日,钉宫理惠
1,布洛妮娅·扎伊切克,8月18日,TetraCalyx
1,布洛妮娅·扎伊切克,8月18日,Hanser
1,布洛妮娅·扎伊切克,8月18日,阿澄佳奈
1,布洛妮娅·扎伊切克,8月18日,花泽香菜
2,德丽莎·阿波卡利斯,3月28日,花玲
2,德丽莎·阿波卡利斯,3月28日,田村由香里


# 字典拆分为多列

In [55]:
df = pd.DataFrame({"id": ["001", "002", "003"],
                   "info": [{"姓名": "琪亚娜·卡斯兰娜", "生日": "12月7日", "外号": "草履虫"},
                            {"姓名": "布洛妮娅·扎伊切克", "生日": "8月18日", "外号": "板鸭"},
                            {"姓名": "德丽莎·阿波卡利斯", "生日": "3月28日", "外号": "德丽傻", "武器": "犹大的誓约"}]
                  })

In [56]:
df

,id,info
0,001,"{'姓名': '琪亚娜·卡斯兰娜', '生日': '12月7日', '外号': '草履虫'}"
1,002,"{'姓名': '布洛妮娅·扎伊切克', '生日': '8月18日', '外号': '板鸭'}"
2,003,"{'姓名': '德丽莎·阿波卡利斯', '生日': '3月28日', '外号': '德丽傻'..."


In [60]:
# 这里的值是一个字典, 而Series接收一个字典的话, 那么字典的key就是索引, value就是值
info_df = df['info'].apply(pd.Series)
print(info_df)

          姓名     生日   外号     武器
0   琪亚娜·卡斯兰娜  12月7日  草履虫    NaN
1  布洛妮娅·扎伊切克  8月18日   板鸭    NaN
2  德丽莎·阿波卡利斯  3月28日  德丽傻  犹大的誓约


In [61]:
df[info_df.columns] = info_df

In [62]:
df

,id,info,姓名,生日,外号,武器
0,001,"{'姓名': '琪亚娜·卡斯兰娜', '生日': '12月7日', '外号': '草履虫'}",琪亚娜·卡斯兰娜,12月7日,草履虫,NaN
1,002,"{'姓名': '布洛妮娅·扎伊切克', '生日': '8月18日', '外号': '板鸭'}",布洛妮娅·扎伊切克,8月18日,板鸭,NaN
2,003,"{'姓名': '德丽莎·阿波卡利斯', '生日': '3月28日', '外号': '德丽傻'...",德丽莎·阿波卡利斯,3月28日,德丽傻,犹大的誓约


# 列转行

In [65]:
# 列转行可以简单地认为是将数据库中的宽表变成一张高表
melt_df = pd.DataFrame({"姓名": ["古明地觉", "雾雨魔理沙", "琪露诺"],
                   "水果": ["草莓", "樱桃", "西瓜"],
                   "星期一": ["70斤", "61斤", "103斤"],
                   "星期二": ["72斤", "60斤", "116斤"],
                   "星期三": ["60斤", "81斤", "153斤"],
                   })
print(melt_df)

      姓名  水果   星期一   星期二   星期三
0   古明地觉  草莓   70斤   72斤   60斤
1  雾雨魔理沙  樱桃   61斤   60斤   81斤
2    琪露诺  西瓜  103斤  116斤  153斤


In [71]:
# id_vars: 第二个参数, 不需要进行列转行的字段, 比如这里的"姓名"和"水果", 在列转行之后会自动进行匹配
# value_vars: 第三个参数, 需要进行列转行的字段
# var_name: 第四个参数, 我们说列转行之后会生成两个列, 第一个列存储的值是"列转行之前的列的列名"，第二个列存储的值是"列转行之前的列的值"。
# 但是生成的两个列总要有列名吧，所以var_name就是生成的第一个列的列名
# value_name: 生成的第二个列的列名
# col_level: 针对于具有二级列名的DataFrame, 这个一般可以不用管

pd.melt(melt_df, id_vars=['姓名', '水果'], value_vars=['星期一', '星期二', '星期三'], var_name=['星期'], value_name='销量')

,姓名,水果,星期,销量
0,古明地觉,草莓,星期一,70斤
1,雾雨魔理沙,樱桃,星期一,61斤
2,琪露诺,西瓜,星期一,103斤
3,古明地觉,草莓,星期二,72斤
4,雾雨魔理沙,樱桃,星期二,60斤
5,琪露诺,西瓜,星期二,116斤
6,古明地觉,草莓,星期三,60斤
7,雾雨魔理沙,樱桃,星期三,81斤
8,琪露诺,西瓜,星期三,153斤


In [74]:
# 使用stack实现melt
melt_df

,姓名,水果,星期一,星期二,星期三
0,古明地觉,草莓,70斤,72斤,60斤
1,雾雨魔理沙,樱桃,61斤,60斤,81斤
2,琪露诺,西瓜,103斤,116斤,153斤


In [75]:
id_vars=["姓名", "水果"]
value_vars=["星期一", "星期二"]
var_name="日期"
value_name="销量"



In [76]:
melt_stack_df = melt_df[id_vars + value_vars]

In [77]:
melt_stack_df

,姓名,水果,星期一,星期二
0,古明地觉,草莓,70斤,72斤
1,雾雨魔理沙,樱桃,61斤,60斤
2,琪露诺,西瓜,103斤,116斤


In [78]:
melt_stack_df.set_index(id_vars)

,,星期一,星期二
姓名,水果,,
古明地觉,草莓,70斤,72斤
雾雨魔理沙,樱桃,61斤,60斤
琪露诺,西瓜,103斤,116斤


In [79]:
melt_stack_df.set_index(id_vars).stack()

姓名     水果     
古明地觉   草莓  星期一     70斤
           星期二     72斤
雾雨魔理沙  樱桃  星期一     61斤
           星期二     60斤
琪露诺    西瓜  星期一    103斤
           星期二    116斤
dtype: object

In [80]:
melt_stack_df.set_index(id_vars).stack().reset_index()

,姓名,水果,level_2,0
0,古明地觉,草莓,星期一,70斤
1,古明地觉,草莓,星期二,72斤
2,雾雨魔理沙,樱桃,星期一,61斤
3,雾雨魔理沙,樱桃,星期二,60斤
4,琪露诺,西瓜,星期一,103斤
5,琪露诺,西瓜,星期二,116斤


In [81]:
res = melt_stack_df.set_index(id_vars).stack().reset_index()

In [82]:
res.columns = id_vars + [var_name, value_name]

In [83]:
res

,姓名,水果,日期,销量
0,古明地觉,草莓,星期一,70斤
1,古明地觉,草莓,星期二,72斤
2,雾雨魔理沙,樱桃,星期一,61斤
3,雾雨魔理沙,樱桃,星期二,60斤
4,琪露诺,西瓜,星期一,103斤
5,琪露诺,西瓜,星期二,116斤


# lreshape 应用

In [86]:
test_list =[['麦克雷', 37, '闪光弹', '66号公路', '战术翻滚', '多拉多', '神射手', '漓江塔'],
['源式', 35, '闪', '尼泊尔', '影', '花村', '斩', '直播骆驼'],
['士兵76', 57, '螺旋飞弹', '多拉多', '生物立场', '伊犁奥斯', '战术目镜', '努巴尼'],]
test_col_list = ['姓名', '年龄', '技能1', '活动地点1', '技能2', '活动地点2', '技能3', '活动地点3']
test_df = pd.DataFrame(test_list, columns=test_col_list)
test_df

,姓名,年龄,技能1,活动地点1,技能2,活动地点2,技能3,活动地点3
0,麦克雷,37,闪光弹,66号公路,战术翻滚,多拉多,神射手,漓江塔
1,源式,35,闪,尼泊尔,影,花村,斩,直播骆驼
2,士兵76,57,螺旋飞弹,多拉多,生物立场,伊犁奥斯,战术目镜,努巴尼


In [88]:
# dropna : boolean, default True
pd.lreshape(data=test_df, groups={'技能': ['技能1', '技能2', '技能3'],
'活动地点': ['活动地点1', '活动地点2', '活动地点3']})

,姓名,年龄,技能,活动地点
0,麦克雷,37,闪光弹,66号公路
1,源式,35,闪,尼泊尔
2,士兵76,57,螺旋飞弹,多拉多
3,麦克雷,37,战术翻滚,多拉多
4,源式,35,影,花村
5,士兵76,57,生物立场,伊犁奥斯
6,麦克雷,37,神射手,漓江塔
7,源式,35,斩,直播骆驼
8,士兵76,57,战术目镜,努巴尼


In [90]:
# 手动实现
test_df['技能'] = test_df[['技能1', '技能2', '技能3']].agg(tuple, axis=1)
test_df['活动地点'] = test_df[['活动地点1', '活动地点2', '活动地点3']].agg(tuple, axis=1)
res_df = test_df.filter(regex=r"(?<!\d)$")
res_df

,姓名,年龄,技能,活动地点
0,麦克雷,37,"(闪光弹, 战术翻滚, 神射手)","(66号公路, 多拉多, 漓江塔)"
1,源式,35,"(闪, 影, 斩)","(尼泊尔, 花村, 直播骆驼)"
2,士兵76,57,"(螺旋飞弹, 生物立场, 战术目镜)","(多拉多, 伊犁奥斯, 努巴尼)"


In [91]:
# 将 "技能" 和 "活动地点" 组合起来得到 "技能-活动地点"。
res_df['技能-活动地点'] = res_df['技能'].combine(res_df['活动地点'], func=lambda x,y: list(zip(x, y)))

/Users/guoqiangzhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
res_df

,姓名,年龄,技能,活动地点,技能-活动地点
0,麦克雷,37,"(闪光弹, 战术翻滚, 神射手)","(66号公路, 多拉多, 漓江塔)","[(闪光弹, 66号公路), (战术翻滚, 多拉多), (神射手, 漓江塔)]"
1,源式,35,"(闪, 影, 斩)","(尼泊尔, 花村, 直播骆驼)","[(闪, 尼泊尔), (影, 花村), (斩, 直播骆驼)]"
2,士兵76,57,"(螺旋飞弹, 生物立场, 战术目镜)","(多拉多, 伊犁奥斯, 努巴尼)","[(螺旋飞弹, 多拉多), (生物立场, 伊犁奥斯), (战术目镜, 努巴尼)]"


In [93]:
res_df.drop(columns=['技能', '活动地点'], inplace=True)
res_df

/Users/guoqiangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,姓名,年龄,技能-活动地点
0,麦克雷,37,"[(闪光弹, 66号公路), (战术翻滚, 多拉多), (神射手, 漓江塔)]"
1,源式,35,"[(闪, 尼泊尔), (影, 花村), (斩, 直播骆驼)]"
2,士兵76,57,"[(螺旋飞弹, 多拉多), (生物立场, 伊犁奥斯), (战术目镜, 努巴尼)]"


In [94]:
res_df = res_df.explode('技能-活动地点')

In [95]:
res_df

,姓名,年龄,技能-活动地点
0,麦克雷,37,"(闪光弹, 66号公路)"
0,麦克雷,37,"(战术翻滚, 多拉多)"
0,麦克雷,37,"(神射手, 漓江塔)"
1,源式,35,"(闪, 尼泊尔)"
1,源式,35,"(影, 花村)"
1,源式,35,"(斩, 直播骆驼)"
2,士兵76,57,"(螺旋飞弹, 多拉多)"
2,士兵76,57,"(生物立场, 伊犁奥斯)"
2,士兵76,57,"(战术目镜, 努巴尼)"


In [96]:
res_df['技能-活动地点'].apply(pd.Series)

,0,1
0,闪光弹,66号公路
0,战术翻滚,多拉多
0,神射手,漓江塔
1,闪,尼泊尔
1,影,花村
1,斩,直播骆驼
2,螺旋飞弹,多拉多
2,生物立场,伊犁奥斯
2,战术目镜,努巴尼


In [98]:
res_df[['技能', '活动地点']] = res_df['技能-活动地点'].apply(pd.Series)

In [99]:
res_df

,姓名,年龄,技能-活动地点,技能,活动地点
0,麦克雷,37,"(闪光弹, 66号公路)",闪光弹,66号公路
0,麦克雷,37,"(战术翻滚, 多拉多)",战术翻滚,多拉多
0,麦克雷,37,"(神射手, 漓江塔)",神射手,漓江塔
1,源式,35,"(闪, 尼泊尔)",闪,尼泊尔
1,源式,35,"(影, 花村)",影,花村
1,源式,35,"(斩, 直播骆驼)",斩,直播骆驼
2,士兵76,57,"(螺旋飞弹, 多拉多)",螺旋飞弹,多拉多
2,士兵76,57,"(生物立场, 伊犁奥斯)",生物立场,伊犁奥斯
2,士兵76,57,"(战术目镜, 努巴尼)",战术目镜,努巴尼
